In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks=pd.read_csv("sp_500_stocks.csv")

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
#Secets.py file
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
base_url="https://sandbox.iexapis.com"

In [6]:
symbol='AAPL'
api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448


In [7]:
data=requests.get(api_url,proxies = {'http':'http://proxy.server:3128'}).json()
#data.status_code
data

{'companyName': 'Apple Inc',
 'marketcap': 2551836687017,
 'week52high': 154,
 'week52low': 89.28,
 'week52highSplitAdjustOnly': 157,
 'week52lowSplitAdjustOnly': 89.81,
 'week52change': 0,
 'sharesOutstanding': 16818674977,
 'float': 0,
 'avg10Volume': 105650985,
 'avg30Volume': 86726802,
 'day200MovingAvg': 130.58,
 'day50MovingAvg': 137.69,
 'employees': 147935,
 'ttmEPS': 4.58,
 'ttmDividendRate': 0.8642400977981873,
 'dividendYield': 0.005937880412047689,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-04',
 'nextEarningsDate': '2021-07-21',
 'peRatio': 32.63231259735231,
 'beta': 1.6071765837422034,
 'maxChangePercent': 56.17180694947671,
 'year5ChangePercent': 5.439464274664557,
 'year2ChangePercent': 1.93673253577481,
 'year1ChangePercent': 0.5306268651951512,
 'ytdChangePercent': 0.11009767235270786,
 'month6ChangePercent': 0.1592681738610955,
 'month3ChangePercent': 0.09335655697756452,
 'month1ChangePercent': 0.12242259278216715,
 'day30ChangePercent': 0.124377069535485

In [8]:
data['year1ChangePercent']

0.5306268651951512

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
print(symbol_strings[0])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA


In [10]:
my_columns=["Ticker" ,"Stock_Price","One_Year_Price_Change","Num_Shares_to_Buy"]
final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call).json()
    for symbol in symbol_string.split(","):
        final_dataframe=final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["price"],
                data[symbol]["stats"]["year1ChangePercent"],
                'N/A'
                
            ],index=my_columns
            ),ignore_index=True
        )
    
    

In [11]:
final_dataframe

,Ticker,Stock_Price,One_Year_Price_Change,Num_Shares_to_Buy
0,A,154.440,0.592303,N/A
1,AAL,19.570,0.676743,N/A
2,AAP,208.420,0.449251,N/A
3,AAPL,142.485,0.539372,N/A
4,ABBV,118.130,0.232378,N/A
...,...,...,...,...
500,YUM,115.200,0.313724,N/A
501,ZBH,150.340,0.146884,N/A
502,ZBRA,535.492,0.939215,N/A
503,ZION,49.840,0.588145,N/A


In [12]:
final_dataframe.sort_values('One_Year_Price_Change',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Stock_Price,One_Year_Price_Change,Num_Shares_to_Buy
0,275,LB,70.440,3.03102,N/A
1,441,TPR,39.100,1.936976,N/A
2,148,DVN,25.524,1.701015,N/A
3,106,COF,156.101,1.632087,N/A
4,410,SIVB,550.790,1.592756,N/A
5,129,DFS,118.700,1.549936,N/A
6,253,IVZ,24.106,1.498696,N/A
7,179,FCX,32.620,1.462715,N/A
8,306,MHK,178.700,1.384646,N/A
9,344,NUE,93.150,1.348977,N/A


In [25]:
def portfolioInput():
    global portfolioSize
    try:
        portfolioSize=float(input("Enter your PortFolio amount "))
    except ValueError:  
        print("Not a number \nPlease try again")
        portfolioSize=input("Enter your PortFolio amount")
portfolioInput()
print(portfolioSize)

Enter your PortFolio amount10000000
10000000.0


In [28]:
positionSize=portfolioSize/len(final_dataframe.index)
print(positionSize)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Num_Shares_to_Buy']=positionSize//final_dataframe.loc[i,'Stock_Price']
final_dataframe

200000.0


D:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Stock_Price,One_Year_Price_Change,Num_Shares_to_Buy
0,275,LB,70.440,3.03102,2839.0
1,441,TPR,39.100,1.936976,5115.0
2,148,DVN,25.524,1.701015,7835.0
3,106,COF,156.101,1.632087,1281.0
4,410,SIVB,550.790,1.592756,363.0
5,129,DFS,118.700,1.549936,1684.0
6,253,IVZ,24.106,1.498696,8296.0
7,179,FCX,32.620,1.462715,6131.0
8,306,MHK,178.700,1.384646,1119.0
9,344,NUE,93.150,1.348977,2147.0


In [31]:
hqmColumns=[
    'Ticker',
    'Price',
    'Num_Shares_to_Buy',
    '1_Year_Price_Return',
    '1_Year_Return_Percentile',
    '6_Month_Price_Return',
    '6_Month_Return_Percentile',
    '3_Month_Price_Return',
    '3_Month_Return_Percentile',
    '1_Month_Price_Return',
    '1_Month_Return_Percentile'   
]

In [32]:
hqmDataframe=pd.DataFrame(columns=hqmColumns)

In [35]:
for symbol_string in symbol_strings:
    batch_api_call=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call).json()
    for symbol in symbol_string.split(","):
        hqmDataframe=hqmDataframe.append(
            pd.Series(
                        [
                            symbol,
                            data[symbol]["price"],
                            'Num_Shares_to_Buy',
                            data[symbol]["stats"]["year1ChangePercent"],
                            'N/A',
                            data[symbol]["stats"]["month6ChangePercent"],
                            'N/A',
                            data[symbol]["stats"]["month3ChangePercent"],
                            'N/A',
                            data[symbol]["stats"]["month1ChangePercent"],
                            'N/A'   
                        ],index=hqmColumns
                    ),ignore_index=True
        )


In [36]:
hqmDataframe

,Ticker,Price,Num_Shares_to_Buy,1_Year_Price_Return,1_Year_Return_Percentile,6_Month_Price_Return,6_Month_Return_Percentile,3_Month_Price_Return,3_Month_Return_Percentile,1_Month_Price_Return,1_Month_Return_Percentile
0,A,154.060,Num_Shares_to_Buy,0.613183,N/A,0.187682,N/A,0.111757,N/A,0.028604,N/A
1,AAL,18.961,Num_Shares_to_Buy,0.693464,N/A,0.264064,N/A,-0.102512,N/A,-0.115177,N/A
2,AAP,210.410,Num_Shares_to_Buy,0.454468,N/A,0.265115,N/A,0.075689,N/A,0.067286,N/A
3,AAPL,147.440,Num_Shares_to_Buy,0.542566,N/A,0.156059,N/A,0.094958,N/A,0.126781,N/A
4,ABBV,115.900,Num_Shares_to_Buy,0.229636,N/A,0.091577,N/A,0.102665,N/A,0.05137,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.770,Num_Shares_to_Buy,0.312652,N/A,0.098226,N/A,-0.014717,N/A,0.01088,N/A
501,ZBH,155.290,Num_Shares_to_Buy,0.14494,N/A,-0.040287,N/A,-0.109721,N/A,-0.031726,N/A
502,ZBRA,512.840,Num_Shares_to_Buy,0.898737,N/A,0.277688,N/A,0.009868,N/A,0.051403,N/A
503,ZION,48.520,Num_Shares_to_Buy,0.573533,N/A,0.02278,N/A,-0.115,N/A,-0.01447,N/A
